In [13]:
# Import the modules
import pandas as pd
from pathlib import Path
import hvplot.pandas
from sklearn.cluster import KMeans

In [17]:
# Read in the CSV file as a Pandas DataFrame
holiday_songs_spotify_df = pd.read_csv(
    Path("data/Holiday_Songs_Spotify.csv")
)

#C:\Users\wsabr\OneDrive\Desktop\project-4\data\Holiday_Songs_Spotify.csv
# Review the DataFrame
holiday_songs_spotify_df.head()

holiday_songs_spotify_df = holiday_songs_spotify_df.assign(Danceable = holiday_songs_spotify_df.danceability > 0.8)
holiday_songs_spotify_df

,Unnamed: 0,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,duration_ms,time_signature,key_mode,playlist_name,playlist_img,track_name,artist_name,album_name,album_img,Danceable
0,1,00IqwkT0PZhJ86PJajRCqk,0.195,0.348,A#,-10.106,major,0.0332,0.820000,0.000000,...,213107,3,A# major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,Silver Bells,Johnny Mathis,Merry Christmas,https://i.scdn.co/image/b878b9e27201163be07e74...,False
1,2,01h424WG38dgY34vkI3Yd0,0.225,0.248,A,-15.871,major,0.0337,0.912000,0.000143,...,183613,4,A major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,White Christmas,Bing Crosby,White Christmas,https://i.scdn.co/image/3bb0daf5f87a737ce67ace...,False
2,3,08BhfyKUXxZrnyHrDavNHP,0.444,0.288,F#,-11.793,major,0.0326,0.911000,0.000007,...,199093,3,F# major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,The Christmas Waltz,Tony Bennett,A Swingin' Christmas Featuring The Count Basie...,https://i.scdn.co/image/96aa4fb09e7fe9d38599c8...,False
3,4,095XSaT8I2uI6Uldj2QrSl,0.687,0.496,A,-4.708,major,0.0339,0.434000,0.001440,...,178680,4,A major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,Stop the Cavalry,Jona Lewie,On The Other Hand There's A Fist (Remastered),https://i.scdn.co/image/50c0ea35cacbf7c5d495c7...,False
4,5,09b2gJR45Pyip2rx9CnXW1,0.477,0.841,F#,-5.172,major,0.0358,0.000165,0.097100,...,203404,4,F# major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,I Don't Want to Go Home for Christmas,Independent Counsel of Funk,I Don't Want to Go Home for Christmas,https://i.scdn.co/image/77eb7c17cafe55037a1ab2...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,163,7qtBirjfexEIOK3xuqYLR3,0.572,0.895,F,-2.654,major,0.0644,0.013400,0.000000,...,147987,4,F major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,Run Run Rudolph,Kelly Clarkson,Wrapped In Red,https://i.scdn.co/image/df6a48dc94245be056b352...,False
163,164,7rrVN7f83kwaz2sGxoFnJY,0.570,0.292,F,-14.408,major,0.0293,0.841000,0.000000,...,207000,4,F major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,Christmas in Our Hearts,Jose Mari Chan,Christmas in Our Hearts,https://i.scdn.co/image/349276cbdfd8857b3975a8...,False
164,165,7t5i1MPAiJZcVRpzQtzzo6,0.838,0.346,G,-8.406,major,0.0471,0.827000,0.011100,...,155347,4,G major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,I Want a Hippopotamus for Christmas (Hippo the...,Gayla Peevey,Christmas Hits,https://i.scdn.co/image/939adaa29092810cc1c449...,True
165,166,7uVnnj80WNEu02JQ3BA3HJ,0.350,0.377,D#,-11.658,major,0.0340,0.353000,0.000000,...,222760,4,D# major,new_holiday_songs,https://mosaic.scdn.co/640/5f79f928a45b878579e...,When a Child Is Born,Johnny Mathis,Christmas Hits,https://i.scdn.co/image/939adaa29092810cc1c449...,False


In [18]:
#drop unneeded columns
holiday_songs_features_df = holiday_songs_spotify_df.drop(['Unnamed: 0','key_mode','time_signature', 'playlist_name', 'playlist_img',
                                                           'track_name', 'artist_name','album_name', 'album_img', 'track_uri'],axis=1)
holiday_songs_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,Danceable
0,0.195,0.348,A#,-10.106,major,0.0332,0.820000,0.000000,0.1260,0.262,166.824,213107,False
1,0.225,0.248,A,-15.871,major,0.0337,0.912000,0.000143,0.4040,0.190,96.013,183613,False
2,0.444,0.288,F#,-11.793,major,0.0326,0.911000,0.000007,0.0987,0.356,108.043,199093,False
3,0.687,0.496,A,-4.708,major,0.0339,0.434000,0.001440,0.0613,0.746,97.575,178680,False
4,0.477,0.841,F#,-5.172,major,0.0358,0.000165,0.097100,0.2570,0.571,119.954,203404,False


In [19]:
#make dummies for key and mode columns
# dummies = df['YEAR'].astype(str).str.get_dummies()
#df = pd.concat([df.drop(columns='YEAR'), dummies], axis=1)

dummies_key = holiday_songs_features_df['key'].astype(str).str.get_dummies()
print(dummies_key)
dummies_mode = holiday_songs_features_df['mode'].astype(str).str.get_dummies()
print(dummies_mode)

     A  A#  B  C  C#  D  D#  E  F  F#  G  G#
0    0   1  0  0   0  0   0  0  0   0  0   0
1    1   0  0  0   0  0   0  0  0   0  0   0
2    0   0  0  0   0  0   0  0  0   1  0   0
3    1   0  0  0   0  0   0  0  0   0  0   0
4    0   0  0  0   0  0   0  0  0   1  0   0
..  ..  .. .. ..  .. ..  .. .. ..  .. ..  ..
162  0   0  0  0   0  0   0  0  1   0  0   0
163  0   0  0  0   0  0   0  0  1   0  0   0
164  0   0  0  0   0  0   0  0  0   0  1   0
165  0   0  0  0   0  0   1  0  0   0  0   0
166  0   0  0  0   0  1   0  0  0   0  0   0

[167 rows x 12 columns]
     major  minor
0        1      0
1        1      0
2        1      0
3        1      0
4        1      0
..     ...    ...
162      1      0
163      1      0
164      1      0
165      1      0
166      1      0

[167 rows x 2 columns]


In [67]:
data = [holiday_songs_features_df, dummies_key, dummies_mode]
holiday_songs_features_df_clean = pd.concat(data, axis=1)
                                            
# holiday_songs_features_df_clean = holiday_songs_features_df_clean.drop(['key', 'mode'], axis=1)

# holiday_songs_features_df_clean.head()

holiday_songs_features_df_clean2 = holiday_songs_features_df_clean.drop(['key', 'mode', 'duration_ms',
                                                                        'A', 'A#', 'B', 'C', 'acousticness',
                                                                         'C#', 'D', 'D#', 'E', 'F', 'F#',
                                                                         'G', 'G#', 'major', 'minor', 
                                                                         'instrumentalness', 'speechiness',
                                                                        'liveness'], axis=1)
holiday_songs_features_df_clean2


,danceability,energy,loudness,valence,tempo,Danceable
0,0.195,0.348,-10.106,0.262,166.824,False
1,0.225,0.248,-15.871,0.190,96.013,False
2,0.444,0.288,-11.793,0.356,108.043,False
3,0.687,0.496,-4.708,0.746,97.575,False
4,0.477,0.841,-5.172,0.571,119.954,False
...,...,...,...,...,...,...
162,0.572,0.895,-2.654,0.922,145.008,False
163,0.570,0.292,-14.408,0.399,72.534,False
164,0.838,0.346,-8.406,0.930,113.988,True
165,0.350,0.377,-11.658,0.277,124.559,False


In [68]:
#  Prepare the data. Use the StandardScaler module and fit_transform function to
from sklearn.preprocessing import StandardScaler
# scale all columns with numerical values
songs_scaled = StandardScaler().fit_transform(holiday_songs_features_df_clean2[["danceability","energy","loudness","valence","tempo"]])
# Diplay the first five rows of the scaled data
songs_scaled[0:5]

array([[-2.14153413, -0.703276  , -0.28167222, -1.08472167,  1.51092846],
       [-1.94575146, -1.14838064, -1.77622892, -1.38123615, -0.71767585],
       [-0.51653799, -0.97033878, -0.71902125, -0.69760555, -0.33906081],
       [ 1.06930163, -0.04452113,  1.11774098,  0.90851454, -0.66851569],
       [-0.30117705,  1.4910899 ,  0.99745055,  0.18781963,  0.035809  ]])

In [69]:
 # Create a a list to store inertia values and the values of k
inertia = []
k = list(range(2, 11))

In [70]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the holiday_songs_spotify DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(songs_scaled)
    inertia.append(k_model.inertia_)

C:\Users\wsabr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\wsabr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\wsabr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\wsabr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\wsabr\anaco

In [71]:
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,2,570.219564
1,3,465.012864
2,4,393.984386
3,5,350.672857
4,6,313.545834


In [72]:
 # Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

In [80]:
#Using k-means with 3 clusters

In [81]:
 # Define the model with 3 clusters
model = KMeans(n_clusters=3, random_state=1)

# Fit the model
model.fit(holiday_songs_features_df_clean2)

# Make predictions
k_3 = model.predict(holiday_songs_features_df_clean2)

# Create a copy of the DataFrame
holiday_songs_features_clean_predictions_df2 = holiday_songs_features_df_clean.copy()

# Add a class column with the labels
holiday_songs_features_clean_predictions_df2['features_segment'] =k_3

C:\Users\wsabr\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [82]:
 # Plot the clusters(3)
holiday_songs_features_clean_predictions_df2.hvplot.scatter(
    x="danceability",
    y="loudness",
    by="features_segment"
)

:NdOverlay   [features_segment]
   :Scatter   [danceability]   (loudness)

In [87]:
# Using k-means with 4 clusters

In [88]:
#  # Define the model with 4 clusters
# model = KMeans(n_clusters=4, random_state=1)

# # Fit the model
# model.fit(holiday_songs_features_df_clean2)

# # Make predictions
# k_4 = model.predict(holiday_songs_features_df_clean2)

# # Create a copy of the DataFrame
# holiday_songs_features_clean_predictions_df2 = holiday_songs_features_df_clean.copy()

# # Add a class column with the labels
# holiday_songs_features_clean_predictions_df2['features_segment'] =k_4

In [89]:
#  # Plot the clusters(4)
# holiday_songs_features_clean_predictions_df2.hvplot.scatter(
#     x="danceability",
#     y="loudness",
#     by="features_segment"
# )

In [58]:
 # Plot the clusters
holiday_songs_features_clean_predictions_df.hvplot.scatter(
    x="energy",
    y="loudness",
    by="features_segment"
)

:NdOverlay   [features_segment]
   :Scatter   [energy]   (loudness)

In [90]:
 # Plot the clusters
holiday_songs_features_clean_predictions_df2.hvplot.scatter(
    x="danceability",
    y="energy",
    by="features_segment"
)

:NdOverlay   [features_segment]
   :Scatter   [danceability]   (energy)

In [92]:
 # Plot the clusters
holiday_songs_features_clean_predictions_df2.hvplot.scatter(
    x="danceability",
    y="loudness",
    by="features_segment"
)

:NdOverlay   [features_segment]
   :Scatter   [danceability]   (loudness)

In [91]:
 # Plot the clusters
holiday_songs_features_clean_predictions_df2.hvplot.scatter(
    x="danceability",
    y="valence",
    by="features_segment"
)

:NdOverlay   [features_segment]
   :Scatter   [danceability]   (valence)

In [38]:
##NOT WORKING...
# import matplotlib.pyplot as plt

# fig = plt.figure() # create new figure
# ax = fig.add_subplot(projection='3d') # add 3d plot

# x = songs_scaled['dancebility']
# y = songs_scaled['tempo']
# z = songs_scaled['energy']
# # make and show plot
# ax = fig.add_subplot(111, projection='3d')
# #ax.scatter(x,y,z)
# plt.show()